In [12]:
# install a SQL kernel.
!pip install ipython-sql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 12.1 MB/s eta 0:00:00


In [13]:
# Load SQL extension
%load_ext sql

### 1. Create database engine and load into SQLite database table

In [14]:
import pandas as pd
from sqlalchemy import create_engine

# List of CSV files to process
csv_files = ['actors.csv', 'customers.csv', 'actsin.csv', 'renting.csv', 'movies.csv']

# Create an SQLite database engine
engine = create_engine('sqlite:///movies_data.db', echo=False)

# Iterate over the CSV files and load them into the database
for csv_file in csv_files:
    # Read the CSV file into a DataFrame
    df = pd.read_csv("Dataset/" + csv_file)
    
    # Extract the table name from the file name (removing the extension)
    table_name = csv_file.split('.')[0]
    
    # Write the DataFrame to an SQLite database table
    df.to_sql(table_name, con=engine, index=False, if_exists='replace')



In [15]:
%%sql sqlite:///movies_data.db

-- Example SQL query
SELECT * FROM movies LIMIT 5;

Done.


movie_id,title,genre,runtime,year_of_releas,renting_price
1,One Night at McCool's,Comedy,93,2001,2.09
2,Swordfish,Drama,99,2001,2.19
3,What Women Want,Comedy,127,2001,2.59
4,Training Day,Drama,122,2001,1.79
5,The Fellowship of the Ring,Science Fiction & Fantasy,178,2001,2.59


## 2. Insight into Customers Preferences

#### Average rating per customer

In [31]:
%%sql 
SELECT customer_id, -- Report the customer_id
      round(avg(rating),2),  -- Report the average rating per customer
      count(rating),  -- Report the number of ratings per customer
      count(*)  -- Report the number of movie rentals per customer
FROM renting
GROUP BY customer_id
HAVING count(*) > 7 -- Select only customers with more than 7 movie rentals
ORDER BY avg(rating) ASC LIMIT 5; -- Order by the average rating in ascending order

 * sqlite:///movies_data.db
Done.


customer_id,"round(avg(rating),2)",count(rating),count(*) -- Report the number of movie rentals per customer
104,6.25,4,8
28,6.71,7,11
111,7.0,3,10
113,7.0,7,15
25,7.2,5,10


#### Sales KPIs in 2018 - Revenues, rentals, and active customers

In [17]:
%%sql 
SELECT 
    SUM(m.renting_price) as revenue, 
    COUNT(*) as num_of_renting, 
    COUNT(DISTINCT r.customer_id) as num_of_customers
FROM renting AS r
LEFT JOIN movies AS m
ON r.movie_id = m.movie_id
-- Only look at movie rentals in 2018
WHERE date_renting BETWEEN '2018-01-01' AND '2018-12-31' ;

 * sqlite:///movies_data.db
Done.


revenue,num_of_renting,num_of_customers
658.02,298,93


#### Income generated by each movie

In [18]:
%%sql
SELECT title AS movie_name, -- Report the income from movie rentals for each movie 
       round(sum(renting_price),2) AS income_movie
FROM
       (SELECT m.title,  
               m.renting_price
       FROM renting AS r
       LEFT JOIN movies AS m
       ON r.movie_id=m.movie_id) AS rm
GROUP BY rm.title
ORDER BY income_movie DESC LIMIT 10; -- Order the result by decreasing income

 * sqlite:///movies_data.db
Done.


movie_name,income_movie
Bridget Jones - The Edge of Reason,37.57
Fair Game,34.68
The Kingdom,31.35
Two for the Money,30.69
Simone,29.59
Django Unchained,29.59
What Women Want,28.49
Monster,27.17
The Two Towers,26.29
A Good Woman,26.01


#### Favourite movies/actors for a group of customers
- **Questions 1**: Which is the favorite mvie on MovieNow for all customers born in the 70?
- **Question 2**: What is the favourite actors from Spain?

In [27]:
%%sql
SELECT m.title as movie_name, 
COUNT(*) AS num_view, -- Report number of views per movie
round(AVG(r.rating),2) as AVG_rating -- Report the average rating per movie
FROM renting AS r
LEFT JOIN customers AS c
ON c.customer_id = r.customer_id
LEFT JOIN movies AS m
ON m.movie_id = r.movie_id
WHERE c.date_of_birth BETWEEN '1970-01-01' AND '1979-12-31'
GROUP BY m.title
HAVING COUNT(*) > 1 -- Remove movies with only one rental
ORDER BY AVG(r.rating) DESC LIMIT 10; -- Order with highest rating first

 * sqlite:///movies_data.db
Done.


movie_name,num_view,AVG_rating
The Fighter,4,10.0
The Fellowship of the Ring,2,10.0
One Night at McCool's,2,10.0
No Country for Old Men,3,10.0
I'm Not There,2,10.0
Django Unchained,4,10.0
Secondhand Lions,4,9.67
Winter's Bone,2,9.5
Fool's Gold,4,9.33
The Help,3,9.0


In [36]:
%%sql
SELECT a.name,  c.gender,
       COUNT(*) AS number_views, 
       AVG(r.rating) AS avg_rating
FROM renting as r
LEFT JOIN customers AS c -- Augment table renting with information about customers 
ON r.customer_id = c.customer_id
LEFT JOIN actsin as ai -- Augment the table renting with the table actsin
ON r.movie_id = ai.movie_id
LEFT JOIN actors as a -- Augment table renting with information about actors
ON ai.actor_id = a.actor_id
WHERE c.country = 'Spain' -- Select only customers from Spain
GROUP BY a.name, c.gender -- For each actor, separately for male and female customers
HAVING AVG(r.rating) IS NOT NULL 
  AND COUNT(*) > 5 -- Report only actors with more than 5 movie rentals
ORDER BY avg_rating DESC, number_views DESC;

 * sqlite:///movies_data.db
Done.


name,gender,number_views,avg_rating
None,male,108,7.71875
None,female,185,7.651685393258427


#### KPIs per country
**Question** -  Sales KPIs - Revenues, rentals, and active customers for each country

In [39]:
%%sql
SELECT 
    c.country,                    -- For each country report
    COUNT(r.movie_id) AS number_renting, -- The number of movie rentals
    round(AVG(r.rating),2) AS average_rating, -- The average rating
    round(SUM(m.renting_price),4) AS revenue         -- The revenue from movie rentals
FROM renting AS r -- Augment the table renting with information about customers
LEFT JOIN customers AS c
ON c.customer_id = r.customer_id
LEFT JOIN movies AS m -- Augment the table renting with information about movies
ON r.movie_id = m.movie_id
WHERE date_renting >= '2019-01-01'  -- Select only records about rentals since the beginning of 2019
GROUP BY c.country;

 * sqlite:///movies_data.db
Done.


country,number_renting,average_rating,revenue
None,1,10.0,1.79
Austria,4,6.0,8.16
Belgium,7,9.0,13.13
Denmark,7,7.71,14.63
France,23,7.53,51.07
Great Britan,9,7.2,17.91
Hungary,11,7.17,25.89
Italy,30,8.24,69.9
Poland,21,8.13,49.29
Slovenia,14,8.0,30.26
